## Latent Dirichlet Allocation

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation as LDA

# http://qwone.com/~jason/20Newsgroups/
dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    categories=['sci.electronics', 'sci.crypt', 'sci.med', 'sci.space']
    # categories=['rec.sport.baseball', 'sci.crypt', 'comp.graphics', 'talk.politics.guns']
)
documents = dataset.data
labels = dataset.target

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(
    max_df=0.95, 
    min_df=2, 
    max_features=1000, 
    stop_words='english',
    strip_accents = 'unicode',
    lowercase = True,
    token_pattern = r'\b[a-zA-Z]{3,}\b'
)
tf = tf_vectorizer.fit_transform(documents)
feature_names = tf_vectorizer.get_feature_names()

# Run LDA
model = LDA(
    n_topics=4, 
    max_iter=5, 
    learning_method='batch', 
    learning_offset=50.,
    random_state=0
).fit(tf)

# Show keywords
no_top_words = 10
for topic_idx, topic in enumerate(model.components_):
    print "Topic %d:" % (topic_idx)
    print " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])

Topic 0:
edu people don like just know com think time good
Topic 1:
space nasa launch earth program data orbit shuttle satellite lunar
Topic 2:
encryption government key use chip clipper privacy information law security
Topic 3:
key use like don bit just used know time good


In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups

categories = ['rec.sport.hockey', 'talk.religion.misc', 'soc.religion.christian']

# training navie bayes
trained_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='train',
    categories=categories
)
X0 = trained_dataset.data
y0 = trained_dataset.target
h_pipeline = Pipeline([
    ('vec', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB(alpha=.1)),
])
%timeit h_pipeline.fit(X0, y0)

# prediction
test_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='test',
    categories=categories
)
X1 = test_dataset.data
y1 = test_dataset.target
test_label_names = test_dataset.target_names
h_pipeline.score(X1, y1)
predicted_y1 = h_pipeline.predict(X1)
print(metrics.classification_report(
    y1, 
    predicted_y1, 
    target_names=test_label_names
))

1 loop, best of 3: 437 ms per loop
                        precision    recall  f1-score   support

      rec.sport.hockey       0.93      0.97      0.95       399
soc.religion.christian       0.69      0.95      0.80       398
    talk.religion.misc       0.93      0.33      0.49       251

           avg / total       0.84      0.81      0.78      1048



In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation as LDA

categories = ['rec.sport.hockey', 'talk.religion.misc', 'soc.religion.christian']

# prepare lda model
all_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    categories=categories
)
X = all_dataset.data
tf_vectorizer = CountVectorizer(
    max_df=0.95, 
    min_df=2, 
    max_features=1000, 
    stop_words='english',
    strip_accents = 'unicode',
    lowercase = True,
    token_pattern = r'\b[a-zA-Z]{3,}\b'
)
tf = tf_vectorizer.fit_transform(X)
lda = LDA(
    n_topics=3, 
    max_iter=5, 
    learning_method='batch', 
    learning_offset=50.,
    random_state=0
).fit(tf)
no_top_words = 10
feature_names = tf_vectorizer.get_feature_names()
for topic_idx, topic in enumerate(lda.components_):
    print "Topic %d:" % (topic_idx)
    print " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])

# training navie bayes
trained_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='train',
    categories=categories
)
X0 = trained_dataset.data
y0 = trained_dataset.target
h_pipeline = Pipeline([
    ('vec', TfidfVectorizer(stop_words='english')),
    ('lda', lda),
    ('clf', MultinomialNB(alpha=.1)),
])
%timeit h_pipeline.fit(X0, y0)

# prediction
test_dataset = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    subset='test',
    categories=categories
)
X1 = test_dataset.data
y1 = test_dataset.target
test_label_names = test_dataset.target_names
h_pipeline.score(X1, y1)
predicted_y1 = h_pipeline.predict(X1)
print(metrics.classification_report(
    y1, 
    predicted_y1, 
    target_names=test_label_names
))

Topic 0:
people god think don know just say like does christian
Topic 1:
team game hockey play season period games year nhl pts
Topic 2:
god jesus christ church law bible john father son spirit
1 loop, best of 3: 3.01 s per loop
                        precision    recall  f1-score   support

      rec.sport.hockey       0.75      0.32      0.45       399
soc.religion.christian       0.43      0.95      0.59       398
    talk.religion.misc       0.00      0.00      0.00       251

           avg / total       0.45      0.48      0.39      1048



/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [7]:
# https://github.com/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation as LDA

pyLDAvis.enable_notebook()

categories = ['rec.sport.hockey', 'talk.religion.misc', 'soc.religion.christian']
newsgroups = fetch_20newsgroups(
    random_state=1, 
    remove=('headers', 'footers', 'quotes'),
    categories=categories
)
docs_raw = newsgroups.data
print(len(docs_raw))
tf_vectorizer = CountVectorizer(
    strip_accents = 'unicode',
    stop_words = 'english',
    lowercase = True,
    token_pattern = r'\b[a-zA-Z]{3,}\b',
    max_df = 0.5, 
    min_df = 10
)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
lda_tf = LDA(    
    n_topics=3, 
    max_iter=5, 
    learning_method='batch', 
    learning_offset=50.,
    random_state=0
)
lda_tf.fit(dtm_tf)
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)
# miscellaneous 就是「各式各樣、五花八門、混雜」的意思

1576


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
1      62.965170        1       1 -0.184427 -0.046176
0      20.725393        1       2  0.202894 -0.034619
2      16.309438        1       3 -0.018468  0.080795, topic_info=     Category         Freq     Term        Total  loglift  logprob
term                                                              
844   Default  1410.000000      god  1410.000000  30.0000  30.0000
2017  Default   454.000000     team   454.000000  29.0000  29.0000
934   Default   350.000000   hockey   350.000000  28.0000  28.0000
812   Default   399.000000     game   399.000000  27.0000  27.0000
1045  Default   705.000000    jesus   705.000000  26.0000  26.0000
1592  Default   207.000000      pts   207.000000  25.0000  25.0000
1490  Default   336.000000     play   336.000000  24.0000  24.0000
1347  Default   227.000000      nhl   227.000000  23.0000  23.0000
1805  Default   289.000000   season   289.000000  22.0000  22.0000
1459  Default   273.000000   period   273.000000  21.0000  21.0000
1119  Default   222.000000      law   222.000000  20.0000  20.0000
813   Default   251.000000    games   251.000000  19.0000  19.0000
1493  Default   202.000000  players   202.000000  18.0000  18.0000
1131  Default   185.000000   league   185.000000  17.0000  17.0000
2242  Default   260.000000     year   260.000000  16.0000  16.0000
340   Default   366.000000   christ   366.000000  15.0000  15.0000
1344  Default   398.000000      new   398.000000  14.0000  14.0000
2199  Default   168.000000      win   168.000000  13.0000  13.0000
2018  Default   151.000000    teams   151.000000  12.0000  12.0000
345   Default   423.000000   church   423.000000  11.0000  11.0000
553   Default   111.000000      det   111.000000  10.0000  10.0000
329   Default   111.000000      chi   111.000000   9.0000   9.0000
225   Default   109.000000      bos   109.000000   8.0000   8.0000
2146  Default   109.000000      van   109.000000   7.0000   7.0000
1595  Default   103.000000     puck   103.000000   6.0000   6.0000
852   Default   487.000000     good   487.000000   5.0000   5.0000
766   Default   128.000000   flyers   128.000000   4.0000   4.0000
557   Default   123.000000  detroit   123.000000   3.0000   3.0000
1482  Default   101.000000      pit   101.000000   2.0000   2.0000
2079  Default   106.000000  toronto   106.000000   1.0000   1.0000
...       ...          ...      ...          ...      ...      ...
1340   Topic3    56.982435      net    91.557321   1.3392  -5.7161
1805   Topic3   125.210396   season   289.298823   0.9760  -4.9288
1646   Topic3    35.599476   recchi    52.449713   1.4259  -6.1865
1159   Topic3    41.202300  lindros    64.292023   1.3685  -6.0403
855    Topic3    89.358542      got   192.384431   1.0466  -5.2661
1490   Topic3   123.553908     play   336.184340   0.8124  -4.9421
1447   Topic3    77.333000     paul   170.137037   1.0249  -5.4107
636    Topic3    74.603401      end   163.443858   1.0291  -5.4466
841    Topic3    54.209013    goals   100.400080   1.1971  -5.7660
564    Topic3    83.681260     didn   203.333570   0.9256  -5.3318
1045   Topic3   180.307796    jesus   705.822024   0.4487  -4.5641
844    Topic3   271.986094      god  1410.423249   0.1675  -4.1531
1492   Topic3    59.568613   player   117.984814   1.1300  -5.6717
1183   Topic3    70.013565     lord   163.183009   0.9672  -5.5101
852    Topic3   128.388728     good   487.244481   0.4797  -4.9037
2062   Topic3   126.101046     time   514.097707   0.4081  -4.9217
2242   Topic3    85.336503     year   260.572943   0.6971  -5.3122
2017   Topic3   109.221142     team   454.779258   0.3870  -5.0654
563    Topic3   106.747056      did   447.462890   0.3803  -5.0883
1070   Topic3   119.972662     just   596.566700   0.2095  -4.9715
340    Topic3    93.346366   christ   366.076866   0.4469  -5.2225
1524   Topic3    73.252340    power   203.458888   0.7919  -5.4649
50

## Spam Detection 

In [5]:
# https://archive.ics.uci.edu/ml/datasets/Spambase
# https://github.com/zhangruiskyline/NLP_DeepLearning#project-1-spam-detector
# picture flow explain lda ~
# https://www.cs.cmu.edu/~./enron/ 
# another email dataset

